In [1]:
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, StackingClassifier
from sklearn.preprocessing import LabelEncoder as LB
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import numpy as np

In [2]:
df = pd.read_csv('games_clean.csv')
df

Presence  Memory  ReleaseDate  OriginalCost  DiscountedCost  \
0        333.0   512.0       2019.0          1.99             0.0   
1        363.0  1000.0       2016.0          2.99             0.0   
2        102.0  1024.0       2019.0          0.99             0.0   
3        269.0  4000.0       2019.0          0.00             0.0   
4        217.0  4000.0       2016.0         11.99             0.0   
...        ...     ...          ...           ...             ...   
2195     161.0   512.0       2014.0          1.99             0.0   
2196     411.0  4000.0       2020.0         11.99             0.0   
2197     249.0  4000.0       2016.0          3.99             0.0   
2198     191.0  1000.0       2019.0         14.99             0.0   
2199  204093.0  4000.0       2019.0         29.99             0.0   

      Achievements  Storage  RatingsBreakdown-Recommended  \
0              0.0    200.0                           2.0   
1              0.0    350.0                           1.0   
2              0.0    122.0                           1.0   
3              0.0    300.0                           2.0   
4             18.0   4000.0                           1.0   
...            ...      ...                           ...   
2195           0.0    150.0                           1.0   
2196           0.0    700.0                           1.0   
2197          20.0    100.0                           1.0   
2198           0.0    800.0                           2.0   
2199          42.0  13000.0                          11.0   

      RatingsBreakdown-Meh  RatingsBreakdown-Exceptional  ...  Web Publishing  \
0                      2.0                           2.0  ...           False   
1                      1.0                           2.0  ...           False   
2                      1.0                           1.0  ...           False   
3                      2.0                           1.0  ...           False   
4                      1.0                           1.0  ...           False   
...                    ...                           ...  ...             ...   
2195                   1.0                           1.0  ...           False   
2196                   1.0                           1.0  ...           False   
2197                   3.0                           1.0  ...           False   
2198                   1.0                           1.0  ...           False   
2199                   6.0                           2.0  ...           False   

     Well-Written  Werewolves  Western  Word Game  World War I  World War II  \
0           False       False    False      False        False         False   
1           False       False    False      False        False         False   
2           False       False    False      False        False         False   
3           False       False    False      False        False         False   
4           False       False    False      False        False         False   
...           ...         ...      ...        ...          ...           ...   
2195        False       False    False      False        False         False   
2196        False       False    False      False        False         False   
2197        False       False    False      False        False         False   
2198        False       False    False      False        False         False   
2199        False       False    False      False        False         False   

      Wrestling  Zombies  eSports  
0         False    False    False  
1         False    False    False  
2         False    False    False  
3         False    False    False  
4         False    False    False  
...         ...      ...      ...  
2195      False    False    False  
2196      False    False    False  
2197      False    False    False  
2198      False    False    False  
2199      False    False    False  

[2200 rows x 469 columns]

In [3]:
for name in df.columns:
    if df[name].isna().sum() > 0:
        df[name].fillna(df[name].mean(), inplace=True)

In [4]:

df.drop(['RatingsBreakdown-Recommended', 'RatingsBreakdown-Meh', 'RatingsBreakdown-Exceptional', 'RatingsBreakdown-Skip', 'Presence'], inplace=True, axis=1)

LE = LB()
class_names = df['Metacritic'].unique()

df.loc[:, 'Metacritic'] = LE.fit_transform(df['Metacritic'])

X = df.loc[:, df.columns != 'Metacritic']
Y = df['Metacritic']


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, shuffle=False)

In [ ]:
####### Bagging classifier
bestNumEstimator = 0
valuescore = 0
for i in range(10,30):
    cv = KFold(n_splits=49, shuffle=False)
    clf = BaggingClassifier(base_estimator=DTC(max_depth=13, criterion='entropy'), 
                            n_estimators=i, random_state=0)
    cv_scores = cross_val_score(clf, X=X,y=Y, cv=cv, scoring='accuracy')  
    meannp = np.mean(cv_scores)
    if valuescore < meannp:
        valuescore = meannp
        bestNumEstimator = i
print('Score of bagging decision tree with 49-fold cross-val. is: ', meannp, 
      ' with a num. of estimators of ', bestNumEstimator)

In [ ]:
####### Random Forest classifier

### K fold cross-val
cv = KFold(n_splits=49, shuffle=False)
clf = RandomForestClassifier(n_estimators = 100, max_depth=13, criterion='entropy')
cv_scores = cross_val_score(clf, X=X,y=Y, cv=cv, scoring='accuracy')  
meannp = np.mean(cv_scores)
print('Score of RFC with 49-fold cross-val. and max_depth of 13 is: ', meannp)

### cross-val 70/30
clffited = clf.fit(X_train, y_train)
scoreCV = accuracy_score(clffited.predict(X_test), y_test)
print('Score of RFC with cross-val 70/30 and max_depth of 13 is: ', scoreCV)


# now we know that 49-fold cross-val is better, we try different num. of estimators to try to improve it
bestnumest = 0
bestestim = 0
for i in range(85,115): # it will take a while...
    clf = RandomForestClassifier(n_estimators = i, max_depth=13, criterion='entropy')
    cv_scores = cross_val_score(clf, X=X,y=Y, cv=cv, scoring='accuracy')
    meannp = np.mean(cv_scores)
    if bestestim < meannp:
        bestestim = meannp
        bestnumest = i

print('Best num of estimators is: ', bestnumest, ' with an score of ', bestestim)

In [ ]:
####### Stacking classifier

estimators = [('rfc', BaggingClassifier(base_estimator=DTC(max_depth=13, criterion='entropy'), 
                            n_estimators=25, random_state=0)),
            ('bc', RandomForestClassifier(n_estimators = 105, max_depth=13, criterion='entropy'))]

clf = StackingClassifier(estimators=estimators)
clffited = clf.fit(X_train, y_train)
scoreCV = accuracy_score(clffited.predict(X_test), y_test)
print('Score of Stacking with cross-val 70/30 and max_depth of 13 is: ', scoreCV)
